# DO NOT ADD OR CREATE ANY NEW CELLS
## DO NOT ADD OR CREATE ANY NEW CELLS
### DO NOT ADD OR CREATE ANY NEW CELLS
#### DO NOT ADD OR CREATE ANY NEW CELLS

### YOU MUST RUN ALL THESE CELLS BEFORE MOVING ON TO YOUR QUESTIONS. IF YOU ENCOUNTER ANY ERRORS JUST RESTART THE KERNEL

In [ ]:
import os
if 'COLAB_GPU' in os.environ or 'COLAB_TPU' in os.environ:
    !pip install ipython-sql==0.5.0 prettytable==3.10.0 --quiet

In [ ]:
import pandas as pd
import requests
from IPython.display import display

In [ ]:
%load_ext sql

In [ ]:
# GitHub raw URL to your .db file
url = "https://github.com/jo5hxxvii/crime_database_practice/raw/refs/heads/main/crime_reports.sqlite"

r = requests.get(url, allow_redirects=True)
if r.status_code == 200:
    with open("crime_reports.sqlite", "wb") as f:
        f.write(r.content)
    print("Database downloaded successfully!")
else:
    print(f"Failed to download, status code: {r.status_code}")


In [ ]:
%sql sqlite:///crime_reports.sqlite

In [ ]:
%%sql tables <<
SELECT name FROM sqlite_master WHERE type='table';

In [ ]:
import sqlite3
dataset = {}
for table in list(tables.DataFrame().name):
    conn = sqlite3.connect("crime_reports.sqlite")
    df = pd.read_sql_query(f'select * from {table}', conn)
    df.columns = [x.lower() for x in df.columns]
    dataset[table] = df
    conn.close()

## Crime Reporting Assessment - Overview and Instructions
-----------------------------------------------------
Goal:
Learner will explore a small crime reporting database (vehicle theft reports) and answer
analytical questions using SQL inside a Jupyter notebook. The notebook uses the %%sql cell magic
to run SQL queries directly against the provided SQLite database.

## **POV:**

You are the data analyst for the **Nigerian Police Force**.  
The organization recently launched the **Central Motor Registry (CMR)** to track stolen vehicle reports and support investigations.

Your task is to write SQL queries that extract information from the CMR database.

Below are the tables in the database and their columns.

---

## **Database Tables and Columns**

- **cars** – id, individual_id, make, model, year, color, plate, created_at  
- **individuals** – id, first_name, last_name, gender, age, phone, email, location_id, created_at  
- **investigators** – id, first_name, last_name, badge_number, jurisdiction_location_id, phone, created_at  
- **locations** – id, name, city, state, latitude, longitude  
- **reports** – id, car_id, reporter_individual_id, investigator_id, status, last_seen_location_id, distance_from_owner_km, reported_at, created_at, notes  

---

## **Important Instructions for Answering the Questions**

### **1. Every answer must begin with this line**
```
%%sql result <<
```
Replace **result** with the correct question identifier.

Examples:

- **Question 1**
  ```
  %%sql q1_result <<
  select * from sales
  ```

- **Question 2**
  ```
  %%sql q2_result <<
  SELECT * FROM orders;
  ```

---

### **2. Write your SQL query on the next line**
Use this structure:

```
%%sql qX_result <<
SELECT ...
FROM ...
WHERE ...
```

---

### **3. Your output must match the expected output exactly**
This includes:

- Column names  
- Letter casing  
- Column order  
- Row order (if specified)  

A mismatch can cause grading to fail.

---

### **4. Failure to follow instructions = score not counted**
If you do not follow the required structure your score will not be recorded.


### Notes:
- Use JOINs, GROUP BY, HAVING, subqueries, CTEs, and window functions (ROW_NUMBER) as requested.

### SAMPLE DATA FROM DATABASE TABLES
Run the cell below to see the sample data from the tables

In [ ]:
print('Cars')
display(dataset['cars'].head())
print('-----------------------------------------------------------------')
print('Locations')
display(dataset['locations'].head())
print('-----------------------------------------------------------------')
print('Individuals')
display(dataset['individuals'].head())
print('-----------------------------------------------------------------')
print('Reports')
display(dataset['reports'].head())
print('-----------------------------------------------------------------')
print('Investigators')
display(dataset['investigators'].head())

## Q1 — Count number of cars per individual and show top 10 owners
Show which individuals own the most cars.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q1_result <<

In [ ]:
%%sql q1_result <<
SELECT 
    i.id AS individual_id,
    i.first_name || ' ' || i.last_name AS owner,
    COUNT(c.id) AS cars_owned
FROM individuals i
INNER JOIN cars c ON i.id = c.individual_id
GROUP BY i.id, i.first_name, i.last_name
ORDER BY cars_owned DESC, individual_id ASC
LIMIT 10;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q1_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_one'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q1_result.DataFrame().equals(dataset['question_one'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q2 — List individuals who own more than 3 cars
Identify people with multiple vehicles.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q2_result <<

In [ ]:
%%sql q2_result <<
SELECT 
    i.id AS individual_id,
    i.first_name,
    i.last_name,
    COUNT(c.id) AS cars_owned
FROM individuals i
INNER JOIN cars c ON i.id = c.individual_id
GROUP BY i.id, i.first_name, i.last_name
HAVING COUNT(c.id) > 3
ORDER BY cars_owned DESC, individual_id ASC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q2_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_two'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q2_result.DataFrame().equals(dataset['question_two'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q3 — Get recent reports with car, owner and report status
See recent car reports along with details of the car, its owner, and where it was last seen order it by report date and return only 20 rows

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q3_result <<

In [ ]:
%%sql q3_result <<
SELECT 
    r.id AS report_id,
    r.reported_at,
    r.status,
    c.plate,
    c.make,
    c.model,
    i.first_name,
    i.last_name,
    l.name AS last_seen_location
FROM reports r
INNER JOIN cars c ON r.car_id = c.id
INNER JOIN individuals i ON c.individual_id = i.id
INNER JOIN locations l ON r.last_seen_location_id = l.id
ORDER BY r.reported_at DESC
LIMIT 20;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q3_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_three'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q3_result.DataFrame().equals(dataset['question_three'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q4 — Count reports per status
Understand how many reports exist for each status.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q4_result <<

In [ ]:
%%sql q4_result <<
SELECT 
    status,
    COUNT(*) AS reports_count
FROM reports
GROUP BY status
ORDER BY reports_count DESC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q4_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_four'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q4_result.DataFrame().equals(dataset['question_four'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q5 — Investigator workload (number of reports assigned)
Show how many reports are assigned to each investigator, along with their jurisdiction.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q5_result <<

In [ ]:
%%sql q5_result <<
SELECT 
    inv.id,
    inv.first_name,
    inv.last_name,
    COUNT(r.id) AS assigned_reports,
    l.name AS jurisdiction
FROM investigators inv
INNER JOIN reports r ON inv.id = r.investigator_id
INNER JOIN locations l ON inv.jurisdiction_location_id = l.id
GROUP BY inv.id, inv.first_name, inv.last_name, l.name
ORDER BY assigned_reports DESC, inv.id ASC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q5_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_five'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q5_result.DataFrame().equals(dataset['question_five'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q6 — Latest report per car
Find the most recent report for each car order it by report date and limit your results to 20 records.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q6_result <<

In [ ]:
%%sql q6_result <<
SELECT 
    r.id AS report_id,
    r.car_id,
    r.reported_at,
    r.status
FROM reports r
INNER JOIN (
    SELECT 
        car_id,
        MAX(reported_at) AS max_reported_at
    FROM reports
    GROUP BY car_id
) latest ON r.car_id = latest.car_id AND r.reported_at = latest.max_reported_at
ORDER BY r.reported_at DESC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q6_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_six'].head(20))

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q6_result.DataFrame().equals(dataset['question_six'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q7 — Average distance from owner by report status
Determine which report statuses are associated with cars being farther from their owners.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q7_result <<

In [ ]:
%%sql q7_result <<
SELECT 
    status,
    ROUND(AVG(distance_from_owner_km), 2) AS avg_distance_km,
    COUNT(*) AS reports_count
FROM reports
GROUP BY status
ORDER BY avg_distance_km DESC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q7_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_seven'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q7_result.DataFrame().equals(dataset['question_seven'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q8 — Owners with cars reported more than once
List individuals whose cars have been reported multiple times.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q8_result <<

In [ ]:
%%sql q8_result <<
SELECT 
    i.id AS owner_id,
    i.first_name,
    i.last_name,
    COUNT(r.id) AS total_reports
FROM individuals i
INNER JOIN cars c ON i.id = c.individual_id
INNER JOIN reports r ON c.id = r.car_id
GROUP BY i.id, i.first_name, i.last_name
HAVING COUNT(r.id) > 1
ORDER BY total_reports DESC, owner_id ASC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q8_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_eight'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q8_result.DataFrame().equals(dataset['question_eight'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q9 — Top 3 most reported cars per last-seen location
Show the cars that are most frequently reported at each location.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q9_result <<

In [ ]:
%%sql q9_result <<
WITH ranked_reports AS (
    SELECT 
        l.name AS location,
        c.plate,
        COUNT(r.id) AS reports_count,
        ROW_NUMBER() OVER (PARTITION BY l.name ORDER BY COUNT(r.id) DESC, c.plate ASC) AS rn
    FROM reports r
    INNER JOIN cars c ON r.car_id = c.id
    INNER JOIN locations l ON r.last_seen_location_id = l.id
    GROUP BY l.name, c.plate
)
SELECT 
    location,
    plate,
    reports_count
FROM ranked_reports
WHERE rn <= 3
ORDER BY location ASC, reports_count DESC, plate ASC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q9_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_nine'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q9_result.DataFrame().equals(dataset['question_nine'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')

## Q10 — First (earliest) report assigned to each investigator by jurisdiction
Identify the earliest report each investigator received in their area.

- **Your column names must match the expected output**
- **clear everything in the cell below and write your sql query there.** 
- Start your answer by typing this exactly as the first line.
```
%%sql q10_result <<

In [ ]:
%%sql q10_result <<
WITH investigator_earliest AS (
    SELECT 
        inv.id AS investigator_id,
        inv.first_name || ' ' || inv.last_name AS investigator,
        r.id AS report_id,
        r.reported_at,
        l.name AS jurisdiction,
        ROW_NUMBER() OVER (PARTITION BY inv.id ORDER BY r.reported_at ASC) AS rn
    FROM investigators inv
    INNER JOIN reports r ON inv.id = r.investigator_id
    INNER JOIN locations l ON inv.jurisdiction_location_id = l.id
)
SELECT 
    investigator_id,
    investigator,
    report_id,
    reported_at,
    jurisdiction
FROM investigator_earliest
WHERE rn = 1
ORDER BY investigator_id ASC;

#### This is the output of your query - run the cell below to see the output of your query

In [ ]:
display(q10_result.DataFrame())

#### Expected Result - Run the cell below to see expected output

In [ ]:
display(dataset['question_ten'])

In [ ]:
#RUN THIS CELL TO CHECK THAT YOUR ANSWER IS CORRECT BEFORE MOVING ON
try:
    assert q10_result.DataFrame().equals(dataset['question_ten'])
    print('✅ Your answer is correct')
except:
    print('❌ Your answer is incorrect, Check your query and try again')